# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}" 
data = requests.get(api_url)
assert(data.status_code == 200)
data = data.json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [10]:
pe_ratio = data["peRatio"]
stock_price = data["latestPrice"]

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [26]:
def chunks_generator(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks_generator(stocks["Ticker"], 100))
symbol_strings = [",".join(symbol_groups[i]) for i in range(len(symbol_groups))]

cols = ["ticker", "stock_price", "pe_ratio", "num_of_shares_to_buy"]
df = pd.DataFrame(columns=cols)
for symbol_string in symbol_strings:
    batch_request_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_request_url)
    assert(data.status_code == 200)
    data = data.json()
    for symbol in symbol_string.split(","):
        df = df.append(pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                pd.NA
            ], 
            index=cols), 
        ignore_index=True)

In [31]:
df.head()

,ticker,stock_price,pe_ratio,num_of_shares_to_buy
0,A,122.810,54.73,<NA>
1,AAL,16.403,-1.18,<NA>
2,AAP,166.680,24.2,<NA>
3,AAPL,136.010,40.65,<NA>
4,ABBV,104.160,23.67,<NA>


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [37]:
df = df.sort_values(by="pe_ratio")
df = df.query('pe_ratio >= 0')
df = df[:50].reset_index(drop=True)
df.head()

,ticker,stock_price,pe_ratio,num_of_shares_to_buy
0,NRG,36.790,2.1,<NA>
1,NLOK,21.534,4.23,<NA>
2,UNM,22.551,4.84,<NA>
3,BIO,588.020,5.2,<NA>
4,AIV,5.177,5.2,<NA>


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy.

In [44]:
portfolio_size = 1_000_000

position_size = portfolio_size / len(df.index)

for row in df.index:
    df.loc[row, "num_of_shares_to_buy"] = math.floor(position_size/df.loc[row, "stock_price"])

df.head()

,ticker,stock_price,pe_ratio,num_of_shares_to_buy
0,NRG,36.790,2.1,543
1,NLOK,21.534,4.23,928
2,UNM,22.551,4.84,886
3,BIO,588.020,5.2,34
4,AIV,5.177,5.2,3863


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [62]:
symbol = "AAPL"

batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}" 
data = requests.get(batch_api_url)
assert(data.status_code == 200)
data = data.json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['EBITDA']

# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['grossProfit']

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [82]:
rv_cols = [
    "Ticker",
    "stock_price",
    "num_of_shares_to_buy",
    "pe_ratio",
    "pe_percentile",
    "pb_ratio",
    "pb_percentile",
    "ps_ratio",
    "ps_percentile",
    "ev_in_mio",
    "ev_ebitda_ratio",
    "ev_ebitda_percentile",
    "ev_gp_ratio",
    "ev_gp_percentile"
]

rv_df = pd.DataFrame(columns=rv_cols)

for symbol_string in symbol_strings:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}" 
    data = requests.get(batch_api_url)
    assert(data.status_code == 200)
    data = data.json()
    for symbol in symbol_string.split(","):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = ev / ebitda
        except TypeError:
            ev_to_ebitda = pd.NA
        try:
            ev_to_gross_profit = ev / gross_profit
        except TypeError:
            ev_to_ebitda = pd.NA
        try:
            ev = ev / 1_000_000
        except TypeError:
            ev_to_ebitda = pd.NA

        rv_df = rv_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    pd.NA,
                    data[symbol]['quote']['peRatio'],
                    pd.NA,
                    data[symbol]['advanced-stats']['priceToBook'],
                    pd.NA,
                    data[symbol]['advanced-stats']['priceToSales'],
                    pd.NA,
                    ev,
                    ev_to_ebitda,
                    pd.NA,
                    ev_to_gross_profit,
                    pd.NA
                ],
                index=rv_cols
            ), ignore_index=True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

In [96]:
for col in rv_df.columns[1:]:
    rv_df[col] = rv_df[col].fillna(rv_df[col].mean())

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [125]:
metrics = {
    "pe_ratio": "pe_percentile", 
    "pb_ratio": "pb_percentile", 
    "ps_ratio": "ps_percentile",
    "ev_ebitda_ratio": "ev_ebitda_percentile",
    "ev_gp_ratio": "ev_gp_percentile"
    }

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric]) / 100

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [109]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = [rv_df.loc[row, metrics[metric]] for metric in metrics.keys()]
    rv_df.loc[row, "rv_score"] = mean(value_percentiles)

rv_df = rv_df.sort_values(by="rv_score", ascending=True)
rv_df.head()

,Ticker,stock_price,num_of_shares_to_buy,pe_ratio,pe_percentile,pb_ratio,pb_percentile,ps_ratio,ps_percentile,ev_in_mio,ev_ebitda_ratio,ev_ebitda_percentile,ev_gp_ratio,ev_gp_percentile,rv_score
459,UNM,22.460,NaN,4.80,15.445545,0.4049,4.950495,0.3823,2.772277,4375.356695,2.359884,3.168317,0.350332,0.396040,5.346535
18,AIG,37.900,NaN,-6.94,10.891089,0.5124,5.742574,0.7008,6.534653,32068.028323,4.068549,3.960396,0.697199,1.386139,5.702970
1,AAL,15.940,NaN,-1.14,14.257426,-1.7100,4.554455,0.3863,2.970297,34377.374504,-5.847348,2.772277,1.353229,5.346535,5.980198
219,HFC,26.498,NaN,-26.44,6.732673,0.7500,7.722772,0.2895,1.584158,5698.482185,5.850122,5.940594,2.109559,11.089109,6.613861
304,MET,47.490,NaN,7.97,16.435644,0.5807,5.940594,0.6232,5.346535,42583.468597,4.413881,4.158416,0.618449,1.188119,6.613861


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [111]:
rv_df = rv_df[:50].reset_index(drop=True)
rv_df

,Ticker,stock_price,num_of_shares_to_buy,pe_ratio,pe_percentile,pb_ratio,pb_percentile,ps_ratio,ps_percentile,ev_in_mio,ev_ebitda_ratio,ev_ebitda_percentile,ev_gp_ratio,ev_gp_percentile,rv_score
0,UNM,22.460,NaN,4.80,15.445545,0.4049,4.950495,0.3823,2.772277,4375.356695,2.359884,3.168317,0.350332,0.396040,5.346535
1,AIG,37.900,NaN,-6.94,10.891089,0.5124,5.742574,0.7008,6.534653,32068.028323,4.068549,3.960396,0.697199,1.386139,5.702970
2,AAL,15.940,NaN,-1.14,14.257426,-1.7100,4.554455,0.3863,2.970297,34377.374504,-5.847348,2.772277,1.353229,5.346535,5.980198
3,HFC,26.498,NaN,-26.44,6.732673,0.7500,7.722772,0.2895,1.584158,5698.482185,5.850122,5.940594,2.109559,11.089109,6.613861
4,MET,47.490,NaN,7.97,16.435644,0.5807,5.940594,0.6232,5.346535,42583.468597,4.413881,4.158416,0.618449,1.188119,6.613861
5,F,8.980,NaN,-17.22,8.316832,1.1500,13.168317,0.2728,1.188119,31109.958563,4.701256,4.752475,2.066405,10.693069,7.623762
6,HIG,49.670,NaN,10.48,18.811881,1.0106,10.891089,0.8561,9.108911,17071.858364,3.947536,3.762376,0.821951,1.980198,8.910891
7,ALL,108.600,NaN,7.46,16.237624,1.3200,17.326733,0.7492,7.128713,33771.289629,2.517101,3.366337,0.721574,1.584158,9.128713
8,BA,223.390,NaN,-28.38,6.534653,-10.8600,2.970297,2.0400,35.346535,162751.850775,-29.827365,1.584158,-215.478515,0.198020,9.326733
9,KSS,40.870,NaN,-59.59,4.950495,1.2700,16.237624,0.3615,2.574257,8681.628048,9.257418,18.613861,1.383459,5.742574,9.623762


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [116]:
portfolio_size = 1_000_000
position_size = portfolio_size / len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, "num_of_shares_to_buy"] = math.floor(position_size / rv_df.loc[row, "stock_price"])

rv_df.head(10)

,Ticker,stock_price,num_of_shares_to_buy,pe_ratio,pe_percentile,pb_ratio,pb_percentile,ps_ratio,ps_percentile,ev_in_mio,ev_ebitda_ratio,ev_ebitda_percentile,ev_gp_ratio,ev_gp_percentile,rv_score
0,UNM,22.460,890.0,4.80,15.445545,0.4049,4.950495,0.3823,2.772277,4375.356695,2.359884,3.168317,0.350332,0.396040,5.346535
1,AIG,37.900,527.0,-6.94,10.891089,0.5124,5.742574,0.7008,6.534653,32068.028323,4.068549,3.960396,0.697199,1.386139,5.702970
2,AAL,15.940,1254.0,-1.14,14.257426,-1.7100,4.554455,0.3863,2.970297,34377.374504,-5.847348,2.772277,1.353229,5.346535,5.980198
3,HFC,26.498,754.0,-26.44,6.732673,0.7500,7.722772,0.2895,1.584158,5698.482185,5.850122,5.940594,2.109559,11.089109,6.613861
4,MET,47.490,421.0,7.97,16.435644,0.5807,5.940594,0.6232,5.346535,42583.468597,4.413881,4.158416,0.618449,1.188119,6.613861
5,F,8.980,2227.0,-17.22,8.316832,1.1500,13.168317,0.2728,1.188119,31109.958563,4.701256,4.752475,2.066405,10.693069,7.623762
6,HIG,49.670,402.0,10.48,18.811881,1.0106,10.891089,0.8561,9.108911,17071.858364,3.947536,3.762376,0.821951,1.980198,8.910891
7,ALL,108.600,184.0,7.46,16.237624,1.3200,17.326733,0.7492,7.128713,33771.289629,2.517101,3.366337,0.721574,1.584158,9.128713
8,BA,223.390,89.0,-28.38,6.534653,-10.8600,2.970297,2.0400,35.346535,162751.850775,-29.827365,1.584158,-215.478515,0.198020,9.326733
9,KSS,40.870,489.0,-59.59,4.950495,1.2700,16.237624,0.3615,2.574257,8681.628048,9.257418,18.613861,1.383459,5.742574,9.623762


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [130]:
writer = pd.ExcelWriter("value_strategy.xlsx", engine="xlsxwriter")
rv_df.to_excel(writer, sheet_name="Value Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [131]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

euro_template = writer.book.add_format(
        {
            'num_format':'0.00 €',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [133]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["stock_price", euro_template],
    "C": ["num_of_shares_to_buy", integer_template],
    "D": ["pe_ratio", float_template],
    "E": ["pe_percentile", percent_template],
    "F": ["pb_ratio", float_template],
    "G": ["pb_percentile", percent_template],
    "H": ["ps_ratio", float_template],
    "I": ["ps_percentile", percent_template],
    "J": ["ev_in_mio", float_template],
    "K": ["ev_ebitda_ratio", float_template],
    "L": ["ev_ebitda_percentile", percent_template],
    "M": ["ev_gp_ratio", float_template],
    "N": ["ev_gp_percentile", percent_template]
    }

for col in column_formats.keys():
    writer.sheets["Value Strategy"].set_column(f"{col}:{col}", 25, column_formats[col][1])
    writer.sheets["Value Strategy"].write(f"{col}1", column_formats[col][0], column_formats[col][1])

writer.save()